# Peer-graded Assignment: Battle of Neighborhoods

## Segmentation & Clustering in the city of Lucknow, Uttar Pradesh, India

 #### Installing the required packages and libraries

In [46]:
import pandas as pd
import numpy as np

# Install Beautiful Soup package, parser and requests package
#!pip install bs4
#!pip install html5lib
!pip install requests

# Import the libraries
import requests
#from bs4 import BeautifulSoup

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#Install packages for geocoders
!conda install -c conda-forge geopy --yes
#!pip install geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#Install Folium package for map rendering
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



#### Get the geographical coordinates of city of Lucknow, India

In [47]:
address = 'Lucknow, India'
geolocator = Nominatim(user_agent="lucknow_explorer")
location = geolocator.geocode(address)
latitude_Lucknow = location.latitude
longitude_Lucknow = location.longitude
print('The geograpical coordinate of Lucknow are {}, {}.'.format(latitude_Lucknow, longitude_Lucknow))

The geograpical coordinate of Lucknow are 26.8381, 80.9346001.


#### Read csv file containing Latitudes & Lontitudes of neighborhoods of Lucknow

In [48]:
lucknow_lat_long = "/resources/labs/DP0701EN/Lucknow_Lat_Long.csv"
lucknow_neighborhoods = pd.read_csv(lucknow_lat_long)
lucknow_neighborhoods

,Neighborhood,Latitude,Longitude
0,Aishbagh,26.838404,80.906328
1,Alambagh,26.812744,80.901280
2,Aliganj,27.492207,79.168962
3,Aliganj Extension,26.889694,80.934943
4,Amausi,26.767357,80.884090
...,...,...,...
59,Vijay Khand,26.857908,80.991251
60,Vikram Khand,26.851657,81.001956
61,Vimal Nagar,26.886420,81.017961
62,Vrindavan Yojna,26.765091,80.957140


#### Visualize Lucknow and its neighborhoods

In [50]:
# create map of Lucknow using latitude and longitude values
map_lucknow = folium.Map(location=[latitude_Lucknow, longitude_Lucknow], zoom_start=11)

# add markers to map
for lat, lng, label in zip(lucknow_neighborhoods['Latitude'], lucknow_neighborhoods['Longitude'], lucknow_neighborhoods['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lucknow)  
    
map_lucknow

#### Utilizing the Foursquare API to explore Toronto neighborhoods and segment them.

##### Define Foursquare credientials and version

In [51]:
CLIENT_ID = 'xxx' # your Foursquare ID
CLIENT_SECRET = 'xxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Exploring the first neighborhood

##### Get neighborhood name, Latitude, Longitude

In [52]:
neighborhood_latitude = lucknow_neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = lucknow_neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = lucknow_neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Aishbagh are 26.838403999999997, 80.906328.


##### Create Get request

In [147]:
radius = 5000
LIMIT = 100
latitude = neighborhood_latitude
longitude = neighborhood_longitude
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

##### Send the Get request and examine the result

In [148]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1c7c1192bf98175b80e34e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Lucknow',
  'headerFullLocation': 'Lucknow',
  'headerLocationGranularity': 'city',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 26.88340404500004,
    'lng': 80.95666629441614},
   'sw': {'lat': 26.793403954999953, 'lng': 80.85598970558387}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d2836d73c7954819718cc9b',
       'name': 'Bara Imam Bara',
       'location': {'address': 'Lucknow',
        'lat': 26.870000526488127,
        'lng': 80.91300472899403,
        'labeledLatLngs': [{'label': 'display',
          'lat': 26.870000526488127,
          'lng':

#### Creating a function to repeat the same process for all the neighborhoods

In [149]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Running the above function on each neighborhood of city of Lucknow and get nearby venues of all the neighborhoods

In [150]:
lucknow_neighborhood_venues = getNearbyVenues(names=lucknow_neighborhoods['Neighborhood'],
                                   latitudes=lucknow_neighborhoods['Latitude'],
                                   longitudes=lucknow_neighborhoods['Longitude']
                                  )

Aishbagh
Alambagh
Aliganj
Aliganj Extension
Amausi
Amber Ganj
Aminabad
Ashiyana Colony
Aurangabad Jagir
Badali Kheda
Badshah Nagar
Bagiamau
Barabirwa
Bharat Nagar
Bhoothnath Market
Budheshwar
Butler Colony
Chandganj
Charbagh
Chinhat
Dilkusha
Dugawan
Golaganj
Golden City
Gomti Nagar
Green City
Haibat Mau Mawaiya
Hazratganj
Hind Nagar
Indira Nagar
Jankipuram
Krishna Nagar
Lohia Vihar
Mahanagar
Manas Vihar
Maulviganj
Meena Market
Mehndiganj
Munshi Pulia
Narayan Nagar
Paper Mill Colony
Pata Nala
Rajaji Puram
Rajendra Nagar
Ratan Khand
River Bank Colony
Saraswati Puram
Sarojini Nagar
Sarvodaya Nagar
Shivpuri Colony
South City
Sunderbagh
Sushant Golf City
Tedhi Pulia
Trans Gomti
United City Colony
Usman Enclave
Vardan Khand
Vigyan Khand
Vijay Khand
Vikram Khand
Vimal Nagar
Vrindavan Yojna
Yaseen Ganj


#### Check the size of resulting dataframe

In [151]:
print(lucknow_neighborhood_venues.shape)
lucknow_neighborhood_venues.head()

(1844, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aishbagh,26.838404,80.906328,Bara Imam Bara,26.870001,80.913005,History Museum
1,Aishbagh,26.838404,80.906328,Sharma Tea Corner,26.847726,80.940747,Tea Room
2,Aishbagh,26.838404,80.906328,The Mughal's Dastarkhwan,26.844244,80.940310,Indian Restaurant
3,Aishbagh,26.838404,80.906328,Rajajipuram E-Block,26.843061,80.880187,Flea Market
4,Aishbagh,26.838404,80.906328,Dastarkhwan,26.852692,80.936927,Indian Restaurant


#### Drop duplicates venues that have same latitude & longitude

In [152]:
#lucknow_neighborhood_venues[lucknow_neighborhood_venues["Venue"]=='Bara Imam Bara']
#lucknow_neighborhood_venues[lucknow_neighborhood_venues["Venue Latitude"]=='26.870001']
bool_series = lucknow_neighborhood_venues.duplicated(["Venue Latitude", "Venue Longitude"])
# display data 
lucknow_neighborhood_venues_unique = lucknow_neighborhood_venues[~bool_series]
lucknow_neighborhood_venues_unique.reset_index(drop = True, inplace = True)
lucknow_neighborhood_venues_unique

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aishbagh,26.838404,80.906328,Bara Imam Bara,26.870001,80.913005,History Museum
1,Aishbagh,26.838404,80.906328,Sharma Tea Corner,26.847726,80.940747,Tea Room
2,Aishbagh,26.838404,80.906328,The Mughal's Dastarkhwan,26.844244,80.940310,Indian Restaurant
3,Aishbagh,26.838404,80.906328,Rajajipuram E-Block,26.843061,80.880187,Flea Market
4,Aishbagh,26.838404,80.906328,Dastarkhwan,26.852692,80.936927,Indian Restaurant
...,...,...,...,...,...,...,...
124,Tedhi Pulia,26.917039,80.954327,Ice N Spice,26.877524,80.970634,Vegetarian / Vegan Restaurant
125,United City Colony,26.922149,80.948533,Pizza Hut,26.896358,80.941474,Pizza Place
126,United City Colony,26.922149,80.948533,Swad,26.901243,80.946290,Breakfast Spot
127,United City Colony,26.922149,80.948533,Aryan Family's Delight,26.881949,80.946905,Restaurant


#### Get number of unique neighborhoods

In [153]:
print('There are {} uniques neighborhoods.'.format(len(lucknow_neighborhood_venues_unique['Neighborhood'].unique())))

There are 26 uniques neighborhoods.


#### Get names of unique venue categories and the count

In [154]:
print('There are {} uniques venue categories.'.format(len(lucknow_neighborhood_venues_unique['Venue Category'].unique())))
lucknow_neighborhood_venues_unique['Venue Category'].unique()

There are 47 uniques venue categories.


array(['History Museum', 'Tea Room', 'Indian Restaurant', 'Flea Market',
       'Ice Cream Shop', 'Café', 'Neighborhood', 'Shopping Mall',
       'Fast Food Restaurant', 'Hotel', 'Hookah Bar', 'Bakery', 'Market',
       'Pizza Place', 'Coffee Shop', 'Sandwich Place', 'Train Station',
       'Multiplex', 'American Restaurant', 'Food Court', 'Shop & Service',
       'Clothing Store', 'Restaurant', 'Breakfast Spot', 'Nightclub',
       'BBQ Joint', 'Chinese Restaurant', 'Park', 'Electronics Store',
       'Department Store', 'Plaza', 'Lounge', 'Fried Chicken Joint',
       'Indian Sweet Shop', 'Grocery Store', 'ATM', 'Platform',
       'Building', 'Bus Station', 'Convenience Store', 'Garden',
       'Food Truck', 'Airport', 'Airport Service', 'Asian Restaurant',
       'Art Gallery', 'Vegetarian / Vegan Restaurant'], dtype=object)

#### Get number of venues for each neighborhood

In [155]:
lucknow_neighborhood_venues_unique.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Aishbagh,44,44,44,44,44,44
Aliganj,1,1,1,1,1,1
Aliganj Extension,8,8,8,8,8,8
Amausi,1,1,1,1,1,1
Aminabad,9,9,9,9,9,9
Ashiyana Colony,4,4,4,4,4,4
Badshah Nagar,20,20,20,20,20,20
Bagiamau,6,6,6,6,6,6
Bharat Nagar,1,1,1,1,1,1


### Analyze each neighborhood

In [156]:
# one hot encoding
lucknow_onehot = pd.get_dummies(lucknow_neighborhood_venues_unique[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
lucknow_onehot['1_Neighborhood'] = lucknow_neighborhood_venues_unique['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [lucknow_onehot.columns[-1]] + list(lucknow_onehot.columns[:-1])
lucknow_onehot = lucknow_onehot[fixed_columns]

lucknow_onehot.head()

,1_Neighborhood,ATM,Airport,Airport Service,American Restaurant,Art Gallery,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot,...,Pizza Place,Platform,Plaza,Restaurant,Sandwich Place,Shop & Service,Shopping Mall,Tea Room,Train Station,Vegetarian / Vegan Restaurant
0,Aishbagh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Aishbagh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,Aishbagh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Aishbagh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Aishbagh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Examining the new dataframe size

In [157]:
lucknow_onehot.shape

(129, 48)

In [158]:
#lucknow_onehot.columns.values
#lucknow_onehot[lucknow_onehot['1_Neighborhood'] == 'Aishbagh']

#### Group rows by neighborhood and by taking mean of the frequency of occurance of each category

In [159]:
#lucknow_grouped = lucknow_onehot.groupby('1_Neighborhood').sum().reset_index()
lucknow_grouped = lucknow_onehot.groupby('1_Neighborhood').mean().reset_index()
lucknow_grouped.head()

,1_Neighborhood,ATM,Airport,Airport Service,American Restaurant,Art Gallery,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot,...,Pizza Place,Platform,Plaza,Restaurant,Sandwich Place,Shop & Service,Shopping Mall,Tea Room,Train Station,Vegetarian / Vegan Restaurant
0,Aishbagh,0.0,0.0,0.0,0.022727,0.0,0.0,0.000000,0.022727,0.000,...,0.045455,0.0,0.0,0.000,0.022727,0.022727,0.068182,0.022727,0.045455,0.0
1,Aliganj,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000,...,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,Aliganj Extension,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.250000,0.125,...,0.250000,0.0,0.0,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,Amausi,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000,...,0.000000,0.0,0.0,1.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,Aminabad,0.0,0.0,0.0,0.000000,0.0,0.0,0.111111,0.000000,0.000,...,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


#### Get dataframe size

In [160]:
lucknow_grouped.shape

(26, 48)

#### Print each neighborhood along with top 5 most common venues

In [161]:
num_top_venues = 5

for hood in lucknow_grouped['1_Neighborhood']:
    print("----"+hood+"----")
    temp = lucknow_grouped[lucknow_grouped['1_Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aishbagh----
                  venue  freq
0     Indian Restaurant  0.18
1  Fast Food Restaurant  0.11
2                 Hotel  0.09
3         Shopping Mall  0.07
4                  Café  0.07


----Aliganj----
            venue  freq
0  Clothing Store   1.0
1             ATM   0.0
2   Grocery Store   0.0
3      Hookah Bar   0.0
4           Hotel   0.0


----Aliganj Extension----
            venue  freq
0     Pizza Place  0.25
1          Bakery  0.25
2     Coffee Shop  0.12
3  Breakfast Spot  0.12
4      Restaurant  0.12


----Amausi----
        venue  freq
0  Restaurant   1.0
1         ATM   0.0
2   Nightclub   0.0
3  Hookah Bar   0.0
4       Hotel   0.0


----Aminabad----
                venue  freq
0               Hotel  0.22
1           Multiplex  0.22
2  Chinese Restaurant  0.11
3                Park  0.11
4   Indian Restaurant  0.11


----Ashiyana Colony----
               venue  freq
0  Electronics Store  0.25
1   Department Store  0.25
2        Pizza Place  0.25
3          

#### Function to sort the venues in descending order

In [162]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create a dataframe and display top 10 venues in each neighborhood

In [163]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = lucknow_grouped['1_Neighborhood']

for ind in np.arange(lucknow_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(lucknow_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()
#neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aishbagh,Indian Restaurant,Fast Food Restaurant,Hotel,Shopping Mall,Café,Train Station,Hookah Bar,Ice Cream Shop,Pizza Place,Multiplex
1,Aliganj,Clothing Store,Vegetarian / Vegan Restaurant,Café,Food Court,Flea Market,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store,Coffee Shop
2,Aliganj Extension,Pizza Place,Bakery,Restaurant,Coffee Shop,Breakfast Spot,Market,Vegetarian / Vegan Restaurant,Café,Flea Market,Fast Food Restaurant
3,Amausi,Restaurant,Vegetarian / Vegan Restaurant,Café,Food Court,Flea Market,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store,Coffee Shop
4,Aminabad,Multiplex,Hotel,BBQ Joint,Park,Nightclub,Indian Restaurant,Chinese Restaurant,Café,Fast Food Restaurant,Electronics Store


## Cluster Neighborhoods

#### run KMeans to cluster the neighborhoods in 5 clusters

In [164]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt # plotting library

In [165]:
# set number of clusters
kclusters = 5

lucknow_grouped_clustering = lucknow_grouped.drop('1_Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lucknow_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10]
kmeans.labels_

array([1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 3, 1, 1, 1, 2, 4, 1, 1, 2,
       1, 1, 2, 0], dtype=int32)

#### Creating a dataframe that contains cluster label and top 10 most common venues

In [166]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

lucknow_merged = lucknow_neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#lucknow_merged = lucknow_merged.join(neighborhoods_venues_sorted, on='Neighborhood')
lucknow_merged = pd.merge(lucknow_merged, neighborhoods_venues_sorted,on='Neighborhood')

lucknow_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aishbagh,26.838404,80.906328,1,Indian Restaurant,Fast Food Restaurant,Hotel,Shopping Mall,Café,Train Station,Hookah Bar,Ice Cream Shop,Pizza Place,Multiplex
1,Aliganj,27.492207,79.168962,1,Clothing Store,Vegetarian / Vegan Restaurant,Café,Food Court,Flea Market,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store,Coffee Shop
2,Aliganj Extension,26.889694,80.934943,2,Pizza Place,Bakery,Restaurant,Coffee Shop,Breakfast Spot,Market,Vegetarian / Vegan Restaurant,Café,Flea Market,Fast Food Restaurant
3,Amausi,26.767357,80.884090,2,Restaurant,Vegetarian / Vegan Restaurant,Café,Food Court,Flea Market,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store,Coffee Shop
4,Aminabad,26.846258,80.926328,1,Multiplex,Hotel,BBQ Joint,Park,Nightclub,Indian Restaurant,Chinese Restaurant,Café,Fast Food Restaurant,Electronics Store


In [167]:
lucknow_merged.shape

(26, 14)

#### Visualize the resulting clusters

In [168]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lucknow_merged['Latitude'], lucknow_merged['Longitude'], lucknow_merged['Neighborhood'], lucknow_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster 1

In [169]:
lucknow_merged.loc[lucknow_merged['Cluster Labels'] == 0, lucknow_merged.columns[[0] + list(range(4, lucknow_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Vimal Nagar,Park,Vegetarian / Vegan Restaurant,Café,Food Court,Flea Market,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store,Coffee Shop


#### Cluster 2

In [170]:
lucknow_merged.loc[lucknow_merged['Cluster Labels'] == 1, lucknow_merged.columns[[0] + list(range(4, lucknow_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aishbagh,Indian Restaurant,Fast Food Restaurant,Hotel,Shopping Mall,Café,Train Station,Hookah Bar,Ice Cream Shop,Pizza Place,Multiplex
1,Aliganj,Clothing Store,Vegetarian / Vegan Restaurant,Café,Food Court,Flea Market,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store,Coffee Shop
4,Aminabad,Multiplex,Hotel,BBQ Joint,Park,Nightclub,Indian Restaurant,Chinese Restaurant,Café,Fast Food Restaurant,Electronics Store
6,Badshah Nagar,Fast Food Restaurant,Shopping Mall,Pizza Place,Café,Indian Restaurant,Fried Chicken Joint,Plaza,Hotel,Multiplex,Department Store
7,Bagiamau,ATM,Grocery Store,Department Store,Platform,Park,Café,Flea Market,Fast Food Restaurant,Electronics Store,Convenience Store
8,Bharat Nagar,Bakery,Vegetarian / Vegan Restaurant,Café,Food Court,Flea Market,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store,Coffee Shop
9,Bhoothnath Market,Hotel,Fast Food Restaurant,Bakery,Vegetarian / Vegan Restaurant,Chinese Restaurant,Food Court,Flea Market,Electronics Store,Department Store,Convenience Store
11,Butler Colony,Food Court,Shopping Mall,Building,Vegetarian / Vegan Restaurant,Café,Flea Market,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store
12,Dilkusha,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Food Court,Flea Market,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store,Coffee Shop
14,Green City,Garden,Indian Restaurant,Convenience Store,Fried Chicken Joint,Food Court,Flea Market,Fast Food Restaurant,Electronics Store,Department Store,Coffee Shop


#### Cluster 3

In [171]:
lucknow_merged.loc[lucknow_merged['Cluster Labels'] == 2, lucknow_merged.columns[[0] + list(range(4, lucknow_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Aliganj Extension,Pizza Place,Bakery,Restaurant,Coffee Shop,Breakfast Spot,Market,Vegetarian / Vegan Restaurant,Café,Flea Market,Fast Food Restaurant
3,Amausi,Restaurant,Vegetarian / Vegan Restaurant,Café,Food Court,Flea Market,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store,Coffee Shop
5,Ashiyana Colony,Café,Electronics Store,Department Store,Pizza Place,Vegetarian / Vegan Restaurant,Food Court,Flea Market,Fast Food Restaurant,Convenience Store,Coffee Shop
10,Budheshwar,Breakfast Spot,Vegetarian / Vegan Restaurant,Café,Food Court,Flea Market,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store,Coffee Shop
17,Jankipuram,Café,Pizza Place,Vegetarian / Vegan Restaurant,Food Court,Flea Market,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store,Coffee Shop
21,South City,Asian Restaurant,Pizza Place,Vegetarian / Vegan Restaurant,Café,Food Court,Flea Market,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store
24,United City Colony,Restaurant,Pizza Place,Breakfast Spot,Vegetarian / Vegan Restaurant,Café,Flea Market,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store


#### Cluster 4

In [172]:
lucknow_merged.loc[lucknow_merged['Cluster Labels'] == 3, lucknow_merged.columns[[0] + list(range(4, lucknow_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Golden City,Bus Station,Café,Food Court,Flea Market,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store,Coffee Shop,Clothing Store


#### Cluster 5

In [173]:
lucknow_merged.loc[lucknow_merged['Cluster Labels'] == 4, lucknow_merged.columns[[0] + list(range(4, lucknow_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lohia Vihar,Train Station,Fried Chicken Joint,Food Truck,Food Court,Flea Market,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store,Coffee Shop
